In [1]:
import pandas as pd
from openai import OpenAI
import json
import re

In [120]:
# Connect to LM Studio
client = OpenAI(base_url="http://26.195.151.129:1234/v1", api_key="lm-studio")

def make_qa(text):
    system_prompt = (
        "Ты помощник по литературе. "
        "Твоя задача: по данному отрывку придумать один вопрос "
        "и ответ без лишней информации. "
        "Отвечай СТРОГО в формате JSON:\n"
        '{ "question": "...", "answer": "..." }\n'
        "Без дополнительного текста, без комментариев."
    )

    user_prompt = (
        "Отрывок:\n"
        f"{text}\n\n"
        "Составь вопрос и ответ."
    )

    response = client.chat.completions.create(
        model="qwen-2.5-7b-base-rag-rl-i1",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        response_format={"type": "text"},
        temperature=0.3,
    )
    content = response.choices[0].message.content.strip()
    
    content = content.strip("`")
    content = content.replace("json", "", 1).strip()
    question, answer = "", ""
    
    try:
        data = json.loads(content)
        question = data.get("question", "").strip()
        answer = data.get("answer", "").strip()
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    else:
        pass
    finally:
        pass
    
    return question, answer

In [124]:
q, a = make_qa("""Гостиная Анны Павловны начала понемногу наполняться. Приехала высшая знать Петербурга, люди самые разнородные по возрастам и характерам, но одинаковые по обществу, в каком все жили; приехала дочь князя Василия, красавица Элен, заехавшая за отцом, чтобы с ним вместе ехать на праздник посланника. Она была в шифре и бальном платье. Приехала и известная, как  la femme la plus séduisante de Pétersbourg, молодая, маленькая княгиня Болконская, прошлую зиму вышедшая замуж и теперь не выезжавшая в большой свет по причине своей беременности, но ездившая еще на небольшие вечера. Приехал князь Ипполит, сын князя Василия, с Мортемаром, которого он представил; приехал и аббат Морио и многие другие.
- Вы не видали еще,- или: - вы не знакомы с та  tante? - говорила Анна Павловна приезжавшим гостям и весьма серьезно подводила их к маленькой старушке в высоких бантах, выплывшей из другой комнаты, как скоро стали приезжать гости, называла их по имени, медленно переводя глаза с гостя на ma tante, и потом отходила.
Все гости совершали обряд приветствования никому не известной, никому не интересной и не нужной тетушки. Анна Павловна с грустным, торжественным участием следила за их приветствиями, молчаливо одобряя их. Ma tante каждому говорила в одних и тех же выражениях о его здоровье, о своем здоровье и о здоровье ее величества, которое нынче было, слава богу, лучше. Все подходившие, из приличия не выказывая поспешности, с чувством облегчения исполненной тяжелой обязанности отходили от старушки, чтоб уж весь вечер ни разу не подойти к ней.
Молодая княгиня Болконская приехала с работой в шитом золотом бархатном мешке. Ее хорошенькая, с чуть черневшимися усиками верхняя губка была коротка по зубам, но тем милее она открывалась и тем еще милее вытягивалась иногда и опускалась на нижнюю. Как это бывает у вполне привлекательных женщин, недостаток ее - короткость губы и полуоткрытый рот - казались ее особенною, собственно ее красотой. Всем было весело смотреть на эту полную здоровья и живости хорошенькую будущую мать, так легко переносившую свое положение. Старикам и скучающим, мрачным молодым людям казалось, что они сами делаются похожи на нее, побыв и поговорив несколько времени с ней. Кто говорил с ней и видел при каждом слове ее светлую улыбочку и блестящие белые зубы, которые виднелись беспрестанно, тот думал, что он особенно нынче любезен. И это думал каждый.
Маленькая княгиня, переваливаясь, маленькими быстрыми шажками обошла стол с рабочею сумочкой на руке и, весело оправляя платье, села на диван, около серебряного самовара, как будто все, что она ни делала, было  partie de plaisir для нее и для всех ее окружавших.
 """)

In [5]:
def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()  # Загружаем весь текст
text_file = "files/books_before/WarAndPeace.txt"
text = load_text(text_file)

In [6]:
question = []
answer = []
context = []

In [129]:
def gen(txt):
  global context
  for i in context:
    if i == txt:
      return 1
  return 0

def sp(text):
  text = re.split(r'\n', text)
  cnt = 0
  ALL = ""
  for tx in text:
    if bool(re.search(r'[a-zA-Z]', tx)):
      continue
    if len(ALL) < 500:
      ALL = ALL + '\n' + tx
      cnt += 1
      continue
    if gen(ALL) == 1:
        continue
    # print(ALL)
    ans = make_qa(tx)
    question.append(ans[0])
    answer.append(ans[1])
    context.append(ALL)
    cnt = 0
    ALL = ""

# context.clear()
# answer.clear()
# question.clear()
sp(text)

In [7]:
data = []
for i in range(len(context)):
  data.append([context[i], answer[i], question[i]])
df = pd.DataFrame(data, columns=['context', 'answer', 'question'])
csv_file = 'output2.csv'
# df.to_csv(csv_file, index=False)

In [2]:
# чистим
df = pd.read_csv("output2.csv")
print(len(df))
df = df[~df["answer"].astype(str).str.contains("неизвестно", case=False, na=False)]
df = df[~df["answer"].astype(str).str.contains("отрывке", case=False, na=False)]
df = df[~df["question"].astype(str).str.contains("отрывке", case=False, na=False)]
df = df[~df["question"].astype(str).str.contains("Что сказал Пьер?", case=False, na=False)]
df = df[~df["question"].astype(str).str.contains("Кто сказал \"можно\"?", case=False, na=False)]
df = df[~df["question"].astype(str).str.contains("Что хотелось посмотреть?", case=False, na=False)]
df = df[df["context"].astype(str).str.split().str.len() >= 3]
len(df)
# excel_file = 'output.xlsx'
# df.to_excel(excel_file, index=False)

2297


2197

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from datasets import Dataset

2025-12-10 13:13:49.683859: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# name = "AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru"
name = "oOundefinedOo/QA-system-T5_RUS"
model = T5ForConditionalGeneration.from_pretrained(name)
tokenizer = T5Tokenizer.from_pretrained(name)

df = df.dropna(subset=["question", "context", "answer"])

# === 2. Обычный train/test split ===
train_df, valid_df = train_test_split(df, test_size=0.1, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)


def preprocess_function(examples):
    inputs = [
        f"question: {q} context: {c}"
        for q, c in zip(examples["question"], examples["context"])
    ]
    model_inputs = tokenizer(
        inputs, max_length=512, truncation=True, padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["answer"], max_length=64, truncation=True, padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tokenized = train_dataset.map(preprocess_function, batched=True)
valid_tokenized = valid_dataset.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    output_dir="./model",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=1e-4,
    num_train_epochs=10,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=valid_tokenized,
)

trainer.train()

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Map:   0%|          | 0/1973 [00:00<?, ? examples/s]

/home/dream_on/miniconda3/envs/tf_env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/220 [00:00<?, ? examples/s]

Step,Training Loss
500,0.516900
1000,0.382000
1500,0.288100
2000,0.215500
2500,0.160300
3000,0.117400
3500,0.084900
4000,0.063300
4500,0.048600


TrainOutput(global_step=4940, training_loss=0.19374200921309623, metrics={'train_runtime': 1183.6994, 'train_samples_per_second': 16.668, 'train_steps_per_second': 4.173, 'total_flos': 1.20147391807488e+16, 'train_loss': 0.19374200921309623, 'epoch': 10.0})

In [7]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# Модель для эмбеддингов
model2 = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Строим эмбеддинги для всех контекстов
context_embeddings = model2.encode(df["context"].tolist(), normalize_embeddings=True)

def get_best_context(question):
    # Эмбеддинг вопроса
    q_emb = model2.encode([question], normalize_embeddings=True)[0]

    # Косинусное сходство — просто скалярное произведение
    scores = np.dot(context_embeddings, q_emb)

    # Индекс лучшего контекста
    best_idx = np.argmax(scores)

    # Возвращаем сам контекст + ответ
    return df.loc[best_idx, "context"], df.loc[best_idx, "answer"], scores[best_idx]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
def generate_answer(question: str, context: str) -> str:
    input_text = f"question: {question} context: {context}"

    # Получаем device модели (cuda или cpu)
    device = next(model.parameters()).device

    enc = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512,
    )

    # Переносим вход на тот же девайс, что и модель
    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=64,
        num_beams=4,
        early_stopping=True,
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [37]:
# question = "Зачем князь Андрей пошел на войну"
question = "Что происходило в Бородинском сражении"
context, answer, score = get_best_context(question)

print("Вопрос:", question)
print("Лучший контекст:", context)
print("Ответ:", generate_answer(question, context))

Вопрос: Что происходило в Бородинском сражении
Лучший контекст: 
Для лакея не может быть великого человека, потому что у лакея свое понятие о величии.
    
    
5 ноября был первый день так называемого Красненского сражения. Перед вечером, когда уже после многих споров и ошибок генералов, зашедших не туда, куда надо; после рассылок адъютантов с противуприказаниями, когда уже стало ясно, что неприятель везде бежит и сражения не может быть и не будет, Кутузов выехал из Красного и поехал в Доброе, куда была переведена в нынешний день главная квартира.
День был ясный, морозный. Кутузов с огромной свитой недовольных им, шушукающихся за ним генералов, верхом на своей жирной белой лошадке ехал к Доброму. По всей дороге толпились, отогреваясь у костров, партии взятых нынешний день французских пленных (их взято было в этот день семь тысяч). Недалеко от Доброго огромная толпа оборванных, обвязанных и укутанных чем попало пленных гудела говором, стоя на дороге подле длинного ряда отпряженных фран

In [38]:
model = T5ForConditionalGeneration.from_pretrained("model/checkpoint-4940")